# BatFlightAnalysis

## Paquetes

In [6]:
import pandas as pd
import numpy as np
import os

from BatFlightAnalysis import read_data, split_data

## Funciones

In [7]:
def curvatura(df):
    # Convertir a arrays de numpy
    x = np.array(df['x'])
    y = np.array(df['y'])
    z = np.array(df['z'])

    # Calcular las derivadas usando diferencias finitas
    dx = np.gradient(x)
    dy = np.gradient(y)
    dz = np.gradient(z)

    ddx = np.gradient(dx)
    ddy = np.gradient(dy)
    ddz = np.gradient(dz)

    # Calcular la curvatura
    curvatura = np.zeros(len(x))

    for i in range(len(x)):
        # Vectores de la primera y segunda derivada
        r_prime = np.array([dx[i], dy[i], dz[i]])
        r_double_prime = np.array([ddx[i], ddy[i], ddz[i]])
        
        # Producto cruzado de r' y r''
        cross_product = np.cross(r_prime, r_double_prime)
        
        # Magnitud de r'
        norm_r_prime = np.linalg.norm(r_prime)
        
        # Curvatura
        curvatura[i] = np.linalg.norm(cross_product) / (norm_r_prime**3)

    # Desviación Estándar de la curvatura
    std_curvatura = np.std(curvatura)

    # Media de la curvatura
    mean_curvatura = np.mean(curvatura)

    return mean_curvatura, std_curvatura

In [8]:
def longitud(df):
    # Convertir a arrays de numpy
    x = np.array(df['x'])
    y = np.array(df['y'])
    z = np.array(df['z'])

    # Longitud de la trayectoria
    longitud = np.zeros(len(x))

    for i in range(1, len(x)):
        longitud[i] = longitud[i-1] + np.sqrt((x[i] - x[i-1])**2 + (y[i] - y[i-1])**2 + (z[i] - z[i-1])**2)

    # Longitud total
    longitud_total = longitud[-1]

    return longitud_total

In [9]:
def var_espacial(df):
    # Convertir a arrays de numpy
    x = np.array(df['x'])
    y = np.array(df['y'])
    z = np.array(df['z'])

    longitud_x = np.max(x) - np.min(x)
    longitud_y = np.max(y) - np.min(y)
    longitud_z = np.max(z) - np.min(z)

    volumen = longitud_x * longitud_y * longitud_z

    return longitud_x, longitud_y, longitud_z, volumen


## Código

### Lectura de datos

In [10]:
data_bef, data_fee, data_aft = read_data('data/output_points_1_no_whiskers.xlsx')

In [11]:
# Split data into flower and bat data; before, during, and after feeding
flower_bef, flower_fee, flower_aft, bat_bef, bat_fee, bat_aft = split_data(data_bef, data_fee, data_aft)

# Show the data
print(flower_fee.head())

      x    y      z
9   281   94  349.0
10  283   98  345.0
11  283  100  341.0
12  283  103  338.0
13  288  103  336.0


### Curvatura

In [12]:
mean_curvatura, std_curvatura = curvatura(flower_fee)
print('Desviación Estándar de la Curvatura:', std_curvatura)
print('Media de la Curvatura:', mean_curvatura)

Desviación Estándar de la Curvatura: 1.8368621915524173
Media de la Curvatura: 0.5475373323457154


### Longitud

In [13]:
# Longitud total
longitud_total = longitud(flower_fee)
print('Longitud Total:', longitud_total)

Longitud Total: 375.82886152263075


### Variación en cada eje

In [14]:
# Variación en cada eje
longitud_x, longitud_y, longitud_z, volumen = var_espacial(flower_fee)

print('Longitud de la trayectoria en el eje x:', longitud_x)
print('Longitud de la trayectoria en el eje y:', longitud_y)
print('Longitud de la trayectoria en el eje z:', longitud_z)
print('Volumen del Espacio:', volumen)

Longitud de la trayectoria en el eje x: 14
Longitud de la trayectoria en el eje y: 15
Longitud de la trayectoria en el eje z: 17.0
Volumen del Espacio: 3570.0


## Iteración sobre archivos

In [20]:
individuo, whiskers, n_vuelo = [], [], []
mean_curvatura, std_curvatura = [], []
longitud_total = []
longitud_x, longitud_y, longitud_z, volumen = [], [], [], []

for n in range(6,10):
    for tipo in ["whiskers", "No whiskers"]:
        # Recorro lo archivos de la capeta Individuo n
        for file in os.listdir(f'data/Individuo {n}/{tipo}'):
            # print("Procesando:", f"Individuo {n}/{tipo}/{file}")

            try:
                data_bef, data_fee, data_aft = read_data(f'data/Individuo {n}/{tipo}/{file}')

                # Split data into flower and bat data; before, during, and after feeding
                flower_bef, flower_fee, flower_aft, bat_bef, bat_fee, bat_aft = split_data(data_bef, data_fee, data_aft)

                # Curvatura
                mean_curv, std_curv = curvatura(flower_fee)
                mean_curvatura.append(mean_curv)
                std_curvatura.append(std_curv)

                # Longitud total
                long = longitud(flower_fee)
                longitud_total.append(long)

                # Variación en cada eje
                long_x, long_y, long_z, vol = var_espacial(flower_fee)
                longitud_x.append(long_x)
                longitud_y.append(long_y)
                longitud_z.append(long_z)
                volumen.append(vol)

                individuo.append(n)
                whiskers.append(tipo)
                n_vuelo.append(file.split('_')[-1].split('.')[0])
            except:
                print("Error en el archivo:", f"Individuo {n}/{tipo}/{file}")

# Creo el conjunto de datos
df = pd.DataFrame({
    'Individuo': individuo,
    'Tipo': tipo,
    'No. Vuelo': n_vuelo,
    'Curvatura Media': mean_curvatura,
    'Desviación Estándar de la Curvatura': std_curvatura,
    'Longitud Total': longitud_total,
    'Longitud en X': longitud_x,
    'Longitud en Y': longitud_y,
    'Longitud en Z': longitud_z,
    'Volumen': volumen
})

# Muestro los datos
display(df.head())

Error en el archivo: Individuo 6/No whiskers/output_points_7.xlsx


C:\Users\matea\AppData\Local\Temp/ipykernel_18656/4145792484.py:31: RuntimeWarning: invalid value encountered in double_scalars
  curvatura[i] = np.linalg.norm(cross_product) / (norm_r_prime**3)


,Individuo,Tipo,No. Vuelo,Curvatura Media,Desviación Estándar de la Curvatura,Longitud Total,Longitud en X,Longitud en Y,Longitud en Z,Volumen
0,6,No whiskers,1,0.319137,0.309382,75.694841,18.0,14.0,19.0,4788.0
1,6,No whiskers,10,0.061655,0.072966,260.010676,54.0,40.0,125.0,270000.0
2,6,No whiskers,11,1.097282,3.454947,80.997540,7.0,21.0,18.0,2646.0
3,6,No whiskers,12,0.128108,0.231968,131.251894,21.0,40.0,93.0,78120.0
4,6,No whiskers,13,0.255117,0.302641,58.087628,10.0,11.0,16.0,1760.0


In [21]:
# Guardo los datos
df.to_excel('data/mediciones.xlsx', index=False)